In [1]:
import warnings
# imporing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.tsa.stattools import acf, adfuller
from statsmodels.graphics.tsaplots import plot_acf
from arch.unitroot import VarianceRatio, ADF
import math
from statsmodels.tsa.arima.model import ARIMA 
from scipy.stats import norm
import numpy as np
from sklearn.metrics import mean_absolute_error 
from pmdarima import auto_arima
warnings.filterwarnings('ignore')


In [2]:
# read csv file
wind_df = pd.read_csv('WindGeneration.csv')
wind_df

,Date,Time,Wind Generation
0,01/01/2014,1,1190.1
1,01/01/2014,2,1186.6
2,01/01/2014,3,1174.3
3,01/01/2014,4,1146.7
4,01/01/2014,5,1156.8
...,...,...,...
8755,31/12/2014,20,996.8
8756,31/12/2014,21,1031.8
8757,31/12/2014,22,1218.0
8758,31/12/2014,23,1312.1


In [3]:
wind_df.head(25)

,Date,Time,Wind Generation
0,01/01/2014,1,1190.1
1,01/01/2014,2,1186.6
2,01/01/2014,3,1174.3
3,01/01/2014,4,1146.7
4,01/01/2014,5,1156.8
5,01/01/2014,6,1189.7
6,01/01/2014,7,1175.4
7,01/01/2014,8,1089.8
8,01/01/2014,9,976.5
9,01/01/2014,10,966.4


In [4]:
#checking for null values
wind_df.isnull().sum()

Date               0
Time               0
Wind Generation    1
dtype: int64

In [5]:
# linear interpolate nan values
wind_df.interpolate(method = 'linear',inplace =True)
wind_df.isnull().sum()

Date               0
Time               0
Wind Generation    0
dtype: int64

In [6]:
# change the date column to datetime
wind_df['Date'] = pd.to_datetime(wind_df['Date'])
# time range from 0-23
wind_df['Time'] = wind_df['Time']-1
# Changing the time format
wind_df['Time'] = wind_df['Time'].apply(lambda x: '{:02}:00:00'.format(x))


In [7]:
# change the formatted time column  to datetime
wind_df['Time'] = pd.to_datetime(wind_df['Time'], format = '%H:%M:%S').dt.time
# combining the date and time column to timestamp column
wind_df['TimeStamp'] = [pd.datetime.combine(wind_df['Date'].iloc[i].date(), wind_df['Time'].iloc[i]) for i in range(wind_df.shape[0])]

In [8]:
wind_df = wind_df.set_index('TimeStamp')
wind_df

,Date,Time,Wind Generation
TimeStamp,,,
2014-01-01 00:00:00,2014-01-01,00:00:00,1190.1
2014-01-01 01:00:00,2014-01-01,01:00:00,1186.6
2014-01-01 02:00:00,2014-01-01,02:00:00,1174.3
2014-01-01 03:00:00,2014-01-01,03:00:00,1146.7
2014-01-01 04:00:00,2014-01-01,04:00:00,1156.8
...,...,...,...
2014-12-31 19:00:00,2014-12-31,19:00:00,996.8
2014-12-31 20:00:00,2014-12-31,20:00:00,1031.8
2014-12-31 21:00:00,2014-12-31,21:00:00,1218.0
